<a href="https://colab.research.google.com/github/jashureddyk/Deep-learning-11554016/blob/main/HW2_The_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [ ]:
# Get the datasets
!wget http://huang.eng.unt.edu/CSCE-5218/test.dat
!wget http://huang.eng.unt.edu/CSCE-5218/train.dat


--2023-02-18 02:47:40--  http://huang.eng.unt.edu/CSCE-5218/test.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2844 (2.8K)
Saving to: ‘test.dat’

test.dat            100%[===================>]   2.78K  --.-KB/s    in 0s      

2023-02-18 02:47:40 (306 MB/s) - ‘test.dat’ saved [2844/2844]

--2023-02-18 02:47:40--  http://huang.eng.unt.edu/CSCE-5218/train.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11244 (11K)
Saving to: ‘train.dat’

train.dat           100%[===================>]  10.98K  --.-KB/s    in 0s      

2023-02-18 02:47:40 (269 MB/s) - ‘train.dat’ saved [11244/11244]



In [ ]:
# Take a peek at the datasets
!head train.dat
!head test.dat

A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
1	1	0	0	0	0	0	0	1	1	0	0	1	0
0	0	1	1	0	1	1	0	0	0	0	0	1	0
0	1	0	1	1	0	1	0	1	1	1	0	1	1
0	0	1	0	0	1	0	1	0	1	1	1	1	0
0	1	0	0	0	0	0	1	1	1	1	1	1	0
0	1	1	1	0	0	0	1	0	1	1	0	1	1
0	1	1	0	0	0	1	0	0	0	0	0	1	0
0	0	0	1	1	0	1	1	1	0	0	0	1	0
0	0	0	0	0	0	1	0	1	0	1	0	1	0
A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
1	1	1	1	0	0	1	1	0	0	0	1	1	0
0	0	0	1	0	0	1	1	0	1	0	0	1	0
0	1	1	1	0	1	1	1	1	0	0	0	1	0
0	1	1	0	1	0	1	1	1	0	1	0	1	0
0	1	0	0	0	1	0	1	0	1	0	0	1	0
0	1	1	0	0	1	1	1	1	1	1	0	1	0
0	1	1	1	0	0	1	1	0	0	0	1	1	0
0	1	0	0	1	0	0	1	1	0	1	1	1	0
1	1	1	1	0	0	1	1	0	0	0	0	1	0


### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [ ]:
import math
import itertools
import re


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')

    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        # Skip any lines that do not contain a tab character
        if not re.search('\t', instance): continue
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input (-1) so that w0 becomes the bias
        instance = [-1] + instance
        data += [instance]
    return data


def dot_product(array1, array2):
    # Ensure that the arrays have the same length
    if len(array1) != len(array2):
        raise ValueError("Arrays must have the same length")
    # Compute the dot product by multiplying corresponding elements and summing
    dot_prod = sum([array1[i] * array2[i] for i in range(len(array1))])
    return dot_prod




import math

def sigmoid(x):
    # Compute the sigmoid function on x using the math library
    sigmoid = 1 / (1 + math.exp(-x))
    return sigmoid


# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    # Add a dummy input so that w0 becomes the bias
    instance = [-1] + instance
    # Ensure that the weight vector and instance have the same length
    if len(weight) < len(instance):
        instance = instance[:len(weight)]
    elif len(weight) > len(instance):
        instance += [0] * (len(weight) - len(instance))
    # Compute the dot product of the weight vector and instance
    dot_prod = sum([weight[i] * instance[i] for i in range(len(weight))])
    # Apply the Heaviside function to the dot product and return the result
    return 1 if dot_prod > 0 else 0


# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    # Compute the output of the model on the instance
    output_val = output(weights, instance)
    # Return the label as 1 if the output is greater than 0.5, else return 0
    if output_val > 0.5:
        return 1
    else:
        return 0



# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # Compute the number of correct predictions on the instances
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    # Compute the accuracy as the percentage of correct predictions
    accuracy = correct * 100 / len(instances)
    return accuracy



# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):

    # Initialize weights to all 0's, but skip the last element
    weights = [0] * (len(instances[0]) - 1)

    for _ in range(epochs):
        for instance in instances:
            # Compute the dot product of the weights and inputs
            in_value = dot_product(weights, instance[:-1])
            # Compute the output using the sigmoid function
            output = sigmoid(in_value)
            # Compute the error
            error = instance[-1] - output
            # Update the weights using the delta rule
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1 - output) * instance[i]

    return weights


## Run it

In [ ]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the follosing code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### TODO Add your answer here (text only)




In the 'train_perceptron' function, we use the 'dot_product' and 'sigmoid' functions to calculate the output of a given 'instance' with the given 'weights'. We then calculate the 'error' as the difference between the predicted 'output' and the actual label of the 'instance'. Using the predict function instead of the 'dot_product' and 'sigmoid' functions would be incorrect as the 'predict' function only returns the sign of the 'dot_product' of the given 'weights' and'instance'.

### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above) 


In [ ]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

for lr in lr_array:
    for tr_size in tr_percent:
        for epochs in num_epochs:
            size =  round(len(instances_tr)*tr_size/100)
            pre_instances = instances_tr[0:size]
            weights = train_perceptron(pre_instances, lr, epochs)
            accuracy = get_accuracy(weights, instances_te)
            print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
                  f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")


#tr: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 65.0
#tr: 40, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  20, lea

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)



A. Do I need to train with all the training dataset to get the highest accuracy with the test dataset?

In my experiment, I found that I do not need to train with all the training dataset to get the highest accuracy with the test dataset. In fact, using only 5% of the training dataset yielded a high accuracy of 79%, which is comparable to the highest accuracy obtained (80%) when using 100% of the training dataset.

B. How do I justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?

In the second run, I used a higher percentage of the training dataset (50%) compared to the first run (20%), but obtained a lower accuracy of 67% compared to the accuracy of 79% obtained in the first run. I think this could be due to overfitting in the second run. When using a higher percentage of the training data, the model may become too specific to the training data, which may cause it to perform poorly on the test data.

C. Can I get higher accuracy with additional hyperparameters (higher than 80.0)?

In my experiment, the highest accuracy obtained was 80% using a learning rate of 0.05, 20 epochs, and 5% of the training data. It is possible that higher accuracy can be obtained by using additional hyperparameters or by tuning the current hyperparameters. However, in my experiment, I did not observe any hyperparameter settings that yielded an accuracy higher than 80%.

D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

In my experiment, I found that training for more epochs did not necessarily improve the accuracy of the model. In fact, using a lower number of epochs (5 or 10) sometimes yielded higher accuracy compared to using a higher number of epochs (20 or 50). Therefore, it is not always worth training for more epochs while keeping all other hyperparameters fixed. It is important to find the sweet spot where the model has learned enough to make accurate predictions, without overfitting to the training data.